# Fintech Sector Portfolio Analysis

(Insert project description)

(Insert table of contents with clickable anchor links)

In [ ]:
import pandas as pd
from pathlib import Path
from MCForecastTools import MCSimulation
import yfinance as yf
import hvplot.pandas
import warnings
warnings.filterwarnings("ignore")

## Collect and Clean Data

### Create a function to generate a daily returns DataFrame for a given list of stocks within a fintech sector

In [ ]:
def get_daily_returns_df(yf_tickers, ticker_names, period):
    """
    Returns a pandas DataFrame where each column are daily returns for the provided stocks. 
    Prints out each step of the data
    
    Parameters
    ----------
    yf_tickers: yfinance.Tickers()
        a yfinance Tickers object containing desired stocks 
    ticker_names: list(str)
        a python list of ticker strings, assumes that the length of the list is equal to the number of stocks in yf_tickers
    period: str
        a python string that correlates to yfinance.Tickers() period parameter (ex: '1d', '2m', '5y', etc.)
        
    Returns
    -------
    pandas.DataFrame
        concatenated DataFrame with each stock's daily return data
    
    """

    stock_dfs = []
    
    # create pandas DataFrames for each stock in the yf ticker object, based on provided time period
    for ticker in ticker_names:
        stock_df = yf_tickers.tickers[ticker].history(period=period)
        
        # verify that the correct data was pulled
        print(f'{ticker} DataFame')
        display(stock_df.head())
        display(stock_df.tail())
        
        # select only the close values for daily returns
        stock_df = stock_df[['Close']]
        
        stock_dfs.append(stock_df)
        
    print('\n')
    
    # concat the list of closing price DataFrames
    daily_returns_df = pd.concat(stock_dfs, axis=1)
    daily_returns_df.columns = ticker_names

    # print out the concatenated DataFrame to verify correct closing price data
    print(f'Closing price DataFrame for {ticker_names}')
    display(daily_returns_df.head())
    display(daily_returns_df.tail())
    print('\n')    

    # convert closing prices to daily returns 
    daily_returns_df = daily_returns_df.pct_change().dropna()

    # print final daily returns DataFrame
    print(f'Daily returns DataFrame for {ticker_names}')
    display(daily_returns_df.head())
    display(daily_returns_df.tail())

    return daily_returns_df

In [ ]:
# definte stock tickers for each sector 
paytech_ticker_names = ['PYPL', 'SQ', 'MA']
lending_ticker_names = ['TREE', 'LC', 'BKI']
banking_ticker_names = ['FISV', 'JKHY', 'FIS']

### Get 5 years of data for Paytech sector - PayPal, Square, and MasterCard

In [ ]:
# create yfinance ticker object for paytech stocks
paytech_tickers = yf.Tickers('pypl sq ma')

# use tickers and 5 year period to get 5 years of paytech stock data 
paytech_daily_returns =  get_daily_returns_df(paytech_tickers, paytech_ticker_names, '5y')

### Get 5 years of data for Lending sector - LendingTree, LendingClub, Black Knight

In [ ]:
# create yfinance ticker object for lending stocks
lending_tickers = yf.Tickers('tree lc bki')

# use tickers and 5 year period to get 5 years of lending stock data 
lending_daily_returns =  get_daily_returns_df(lending_tickers, lending_ticker_names, '5y')

### Get 5 years of data for Banking sector - Fiserv, Jack Henry & Associates, FIS (Fidelity National Information Services)

In [ ]:
# create yfinance ticker object for banking stocks
banking_tickers = yf.Tickers('fisv jkhy fis')

# use tickers and 5 year period to get 5 years of lending stock data 
banking_daily_returns = get_daily_returns_df(banking_tickers, banking_ticker_names, '5y')

## Data Analysis and Calculations

### Plot the daily returns in composite graphs for each sector

In [ ]:
paytech_daily_returns['PYPL'].hvplot(
    title='PayPal Daily Returns 2017-2022',
    frame_height=500,
    frame_width=900
) + paytech_daily_returns['SQ'].hvplot(
    title='Square Daily Returns 2017-2022',
    frame_height=500,
    frame_width=900
) + paytech_daily_returns['MA'].hvplot(
    title='MasterCard Daily Returns 2017-2022',
    frame_height=500,
    frame_width=900
)

In [ ]:
lending_daily_returns['TREE'].hvplot(
    title='LendingTree Daily Returns 2017-2022',
    frame_height=500,
    frame_width=900
) + lending_daily_returns['LC'].hvplot(
    title='LendingClub Daily Returns 2017-2022',
    frame_height=500,
    frame_width=900
) + lending_daily_returns['BKI'].hvplot(
    title='Black Knight Daily Returns 2017-2022',
    frame_height=500,
    frame_width=900
)

In [ ]:
banking_daily_returns['FISV'].hvplot(
    title='Fiserv Daily Returns 2017-2022',
    frame_height=500,
    frame_width=900
) + banking_daily_returns['JKHY'].hvplot(
    title='Jack Henry & Associates Daily Returns 2017-2022',
    frame_height=500,
    frame_width=900
) + banking_daily_returns['FIS'].hvplot(
    title='FIS Daily Returns 2017-2022',
    frame_height=500,
    frame_width=900
)

## Calculate and plot the cumulative returns of each sector 

In [ ]:
paytech_cumulative_returns = (1 + paytech_daily_returns).cumprod()
lending_cumulative_returns = (1 + lending_daily_returns).cumprod()
banking_cumulative_returns = (1 + banking_daily_returns).cumprod()

In [ ]:
paytech_cumulative_plot = paytech_cumulative_returns.hvplot(
    title='Paytech (PayPal, Square, MasterCard) Cumulative Returns 2017-2022',
    frame_width=1000,
    frame_height=600,
    group_label='Stock Ticker'
)

paytech_cumulative_plot

In [ ]:
lending_cumulative_plot = lending_cumulative_returns.hvplot(
    title='Lending (LendingTree, LendingClub, Black Knight) Cumulative Returns 2017-2022',
    frame_width=1000,
    frame_height=600,
    group_label='Stock Ticker'
)

lending_cumulative_plot

In [ ]:
banking_cumulative_plot = banking_cumulative_returns.hvplot(
    title='Banking (Fiserv, Jack Henry & Associates, FIS) Cumulative Returns 2017-2022',
    frame_width=1000,
    frame_height=600,
    group_label='Stock Ticker'
)

banking_cumulative_plot

In [ ]:
# combine the sector cumulative return plots in an overlay plot
cumulative_overlay_plot = paytech_cumulative_plot * lending_cumulative_plot * banking_cumulative_plot
cumulative_overlay_plot.opts(title='Cumulative Returns for Paytech, Lending, and Banking Sectors 2017-2022')

## Calculate and plot 21-Day Rolling Average and Standard Deviation for each sector

In [ ]:
paytech_rolling_mean = paytech_daily_returns.rolling(window=21).mean().hvplot(
    frame_width=1000,
    frame_height=600, 
    title = "Paytech Daily Prices - 21-Day Rolling Average")

paytech_rolling_stdev = paytech_daily_returns.rolling(window=21).std().hvplot( 
    frame_width=1000,
    frame_height=600, 
    title = "Paytech Daily Prices - 21-Day Rolling Standard Deviation")

display(paytech_rolling_mean)
display(paytech_rolling_stdev)

In [ ]:
lending_rolling_mean = lending_daily_returns.rolling(window=21).mean().hvplot(
    frame_width=1000,
    frame_height=600, 
    title = "Lending Daily Prices - 21-Day Rolling Average")

lending_rolling_stdev = lending_daily_returns.rolling(window=21).std().hvplot( 
    frame_width=1000,
    frame_height=600, 
    title = "Lending Daily Prices - 21-Day Rolling Standard Deviation")

display(lending_rolling_mean)
display(lending_rolling_stdev)

In [ ]:
banking_rolling_mean = banking_daily_returns.rolling(window=21).mean().hvplot(
    frame_width=1000,
    frame_height=600, 
    title = "Lending Daily Prices - 21-Day Rolling Average")

banking_rolling_stdev = banking_daily_returns.rolling(window=21).std().hvplot( 
    frame_width=1000,
    frame_height=600, 
    title = "Lending Daily Prices - 21-Day Rolling Standard Deviation")

display(banking_rolling_mean)
display(banking_rolling_stdev)

# Monte Carlo Simulations

## Reformat sector DataFrames to be usable by MCSimulation module

In [ ]:
def get_data_for_mcsim(yf_tickers, ticker_names, period):
    """
    Returns a DataFrame with multi-level column names as required by the MCSimulation module 
    
    Parameters
    ----------
    yf_tickers: yfinance.Tickers()
        a yfinance Tickers object containing desired stocks
    ticker_names: list(str)
        a python list of ticker strings, assumes that the length of the list is equal to the number of stocks in yf_tickers
    period: str
        a python string that correlates to yfinance.Tickers() period parameter (ex: '1d', '2m', '5y', etc.)
        
    Returns
    -------
    pandas.DataFrame
        concatenated DataFrame where the first level of columns is the stock ticker and the second is each stock's OHLCV data
    
    """
    
    stock_dfs = []
    
    for ticker in ticker_names:
        # add the full DataFrame, except for extraneous columns, to list for concat
        stock_dfs.append(yf_tickers.tickers[ticker].history(period=period).drop(['Dividends', 'Stock Splits'], axis=1))
    
    mc_data_df = pd.concat(stock_dfs, axis=1, keys=ticker_names)

    # rename Close to lower case so MCSimulation can select it 
    mc_data_df = mc_data_df.rename(columns={'Close': 'close'})
    
    display(mc_data_df.head())
    display(mc_data_df.tail())
    
    return mc_data_df

In [ ]:
# format data for Paytech sector
paytech_df = get_data_for_mcsim(paytech_tickers, paytech_ticker_names, '5y')

In [ ]:
# format data for lending sector
lending_df = get_data_for_mcsim(lending_tickers, lending_ticker_names, '5y')

In [ ]:
banking_df = get_data_for_mcsim(banking_tickers, banking_ticker_names, '5y')

In [ ]:
# create MCSimulation object
mc_test_paytech = MCSimulation(
    portfolio_data=paytech_df,
    weights=[.33, .33, .33],
    num_simulation=500,
    num_trading_days=252*5
)

In [ ]:
mc_test_paytech.calc_cumulative_return()

In [ ]:
mc_test_paytech.plot_simulation()

In [ ]:
mc_test_banking = MCSimulation(
    portfolio_data=banking_df,
    weights=[.33, .33, .33],
    num_simulation=500,
    num_trading_days=252*5
)

In [ ]:
mc_test_banking.calc_cumulative_return()

In [ ]:
mc_test_banking.plot_simulation()

In [ ]:
mc_test_lending = MCSimulation(
    portfolio_data=lending_df,
    weights=[.33, .33, .33],
    num_simulation=500,
    num_trading_days=252*5
)

In [ ]:
mc_test_lending.calc_cumulative_return()

In [ ]:
mc_test_lending.plot_simulation()

In [ ]:
#

In [ ]:
mc_test_paytech2 = MCSimulation(
    portfolio_data=paytech_df,
    weights=[.50, .20, .30],
    num_simulation=500,
    num_trading_days=252*5
)

In [ ]:
mc_test_paytech2.calc_cumulative_return()

In [ ]:
mc_test_paytech2.plot_simulation()

In [ ]:
mc_test_banking2 = MCSimulation(
    portfolio_data=banking_df,
    weights=[.50, .20, .30],
    num_simulation=500,
    num_trading_days=252*5
)

In [ ]:
mc_test_banking2.calc_cumulative_return()

In [ ]:
mc_test_banking2.plot_simulation()

In [ ]:
mc_test_lending2 = MCSimulation(
    portfolio_data=lending_df,
    weights=[.50,.20, .30],
    num_simulation=500,
    num_trading_days=252*5
)

In [ ]:
mc_test_lending2.calc_cumulative_return()

In [ ]:
mc_test_lending2.plot_simulation()

In [ ]:
mc_1y_test_paytech2 = MCSimulation(
    portfolio_data=paytech_df,
    weights=[.33, .33, .33],
    num_simulation=500,
    num_trading_days=252*1
)

In [ ]:
mc_1y_test_paytech2.calc_cumulative_return()

In [ ]:
mc_1y_test_paytech2.plot_simulation()

In [ ]:
mc_1y_test_banking2 = MCSimulation(
    portfolio_data=banking_df,
    weights=[.33, .33, .33],
    num_simulation=500,
    num_trading_days=252*1
)

In [ ]:
mc_1y_test_banking2.calc_cumulative_return()

In [ ]:
mc_1y_test_banking2.plot_simulation()

In [ ]:
mc_1y_test_lending2 = MCSimulation(
    portfolio_data=lending_df,
    weights=[.33, .33, .33],
    num_simulation=500,
    num_trading_days=252*1
)

In [ ]:
mc_1y_test_lending2.calc_cumulative_return()

In [ ]:
mc_test_lending2.plot_simulation()